# Title

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * a
  

### Function  `output_from` <a id="output_from"></a>

The default input of a step is the output from its previous step but you can use function `output_from` to specify the input of a step asthe output from one or more other steps. For example,

In [12]:
%run
[step_10]
output: 'a.txt'
_output.touch()

[step_20]
output: 'b.txt'
print(f'input of step {step_name} is {step_input}')
_output.touch()

[step_30]
input:  output_from(['step_10', 'step_20']), 'c.txt'
print(f'input of step {step_name} is {step_input} with sources {step_input.sources}')
print(f'Output of step_20 is {step_input["step_20"]}')

,step,Workflow ID f3820d6d856346af,Index #5,completed Ran for < 5 seconds


input of step step_20 is a.txt
input of step step_30 is a.txt b.txt c.txt with sources ['step_10', 'step_20', 'step_30']
Output of step_20 is b.txt


In this example, `output_from(['step_10', 'step_20'])` takes the output of steps `step_10` and `step_20` as input. The `sources` of these input are `step_10` and `step_20` respectively. In a process-oriented workflow, `output_from(['step_10', 'step_20'])` can be simplified as `output_from([10, 20])` (integers).

You can override the `sources` of input files with keyword arguments

In [13]:
%run
[step_10]
output: 'a.txt'
print(f'input of step {step_name} is {step_input}')
_output.touch()

[step_20]
output: 'b.txt'
print(f'input of step {step_name} is {step_input}')
_output.touch()

[step_30]
input:  output_from(10), s20=output_from(20), s30='c.txt'
print(f'input of step {step_name} is {step_input} with sources {step_input.sources}')


,step,Workflow ID 5b1a7c8203ac5355,Index #6,completed Ran for < 5 seconds


input of step step_10 is 
input of step step_20 is a.txt
input of step step_30 is a.txt b.txt c.txt with sources ['step_10', 's20', 's30']


As you can learn later, output from other steps can also have their own sources. In this case, the `sources` of the output is carried over.

In [15]:
%run
[step_10]
output: output='out.txt', summary='summary.txt'
_output.touch()

[step_30]
input:  output_from(10), 'c.txt'
print(f'input of step {step_name} is {step_input} with sources {step_input.sources}')

,step,Workflow ID b2d5163796f0f4f8,Index #8,completed Ran for < 5 seconds


input of step step_30 is out.txt summary.txt c.txt with sources ['output', 'summary', 'step_30']


Now, if you are only interested to the `summary` part of the output of `step_10`, you can use `['summary']` to get a subset of the output from `output_from(10)`:

In [18]:
%run
[step_10]
output: output='out.txt', summary='summary.txt'
_output.touch()

[step_30]
input:  output_from(10)['summary'], 'c.txt'
print(f'input of step {step_name} is {step_input} with sources {step_input.sources}')

,step,Workflow ID b2d5163796f0f4f8,Index #11,completed Ran for < 5 seconds


input of step step_30 is summary.txt c.txt with sources ['summary', 'step_30']


When you use keyword argument to specify all or parts of the outputs, the `sources` are overridden

In [22]:
%run
[step_10]
output: a='a.txt', b='b.txt'
_output.touch()

[step_20]
output: c='c.txt', d='d.txt'
_output.touch()

[step_30]
input:  s10=output_from(10), c=output_from(20)["c"]
print(f'input of step {step_name} is {step_input} with sources {step_input.sources}')

,step,Workflow ID 84533aaef4b9d9dd,Index #15,completed Ran for 0 sec


input of step step_30 is a.txt b.txt c.txt with sources ['s10', 's10', 'c']


Note that both source `a` and `b` from `output_from(10)` are overriden by `s10` so you can no longer differentiate sources `a` and `b` from `output_from(10)`.

## Further reading

* 